<b><font size="6">New York City Taxi Fare Prediction Playground Competition</br>
    Creating a raw feather file</font></b>

**PDS ILV bds19: Summer School**  
**Data Science & Business Analytics**

Autor: Thomas Schrittwieser, DS191004  
Datum: 2020-07-20

---

In [5]:
import pandas as pd 
import os
from tqdm import tqdm

## Configuration

In [6]:
# Configuration

write_to_feather = 0 # If you want to write to feather-file at the end, choose 1

# name of the input csv-file
csv_file = '../data/new-york-city-taxi-fare-prediction/train.csv'

# name of the output feather-file
feather_file = '../data/new-york-city-taxi-fare-prediction/nyc_taxi_data_raw.feather'

# Read csv-file

In [7]:
%%time

#Find out the exact number of rows from the file
with open(csv_file) as file:
    n_rows = len(file.readlines())
print (f'Exact number of rows: {n_rows}')

Exact number of rows: 55423857
Wall time: 31.1 s


In [8]:
# Peep at the training file header
df_tmp = pd.read_csv(csv_file, nrows=5)
df_tmp.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [9]:
# Set columns to most suitable type to optimize for memory usage
datatypes = {'fare_amount': 'float32', 
             'pickup_datetime': 'str', 
             'pickup_longitude': 'float32',
             'pickup_latitude': 'float32',
             'dropoff_longitude': 'float32',
             'dropoff_latitude': 'float32',
             'passenger_count': 'uint8'}

cols = list(datatypes.keys())

In [10]:
# Specify the chunksize for the import of the csv-file
chunksize = 5_000_000 # 5 million rows at one go

In [11]:
%%time
df_list = [] # list to hold the batch dataframe

for df_chunk in tqdm(pd.read_csv(csv_file, usecols=cols, dtype=datatypes, chunksize=chunksize)):
     
    # Neat trick from https://www.kaggle.com/btyuhas/bayesian-optimization-with-xgboost
    # Using parse_dates would be much slower!
    df_chunk['pickup_datetime'] = df_chunk['pickup_datetime'].str.slice(0, 16)
    df_chunk['pickup_datetime'] = pd.to_datetime(df_chunk['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
    
    # Can process each chunk of dataframe here
    # clean_data(), feature_engineer(),fit()
    
    # Alternatively, append the chunk to list and merge all
    df_list.append(df_chunk) 

12it [01:45,  8.83s/it]

Wall time: 1min 45s


In [12]:
# Merge all dataframes into one dataframe
df = pd.concat(df_list)

# Delete the dataframe df_list to release memory
del df_list

# See what we have loaded
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55423856 entries, 0 to 55423855
Data columns (total 7 columns):
 #   Column             Dtype              
---  ------             -----              
 0   fare_amount        float32            
 1   pickup_datetime    datetime64[ns, UTC]
 2   pickup_longitude   float32            
 3   pickup_latitude    float32            
 4   dropoff_longitude  float32            
 5   dropoff_latitude   float32            
 6   passenger_count    uint8              
dtypes: datetime64[ns, UTC](1), float32(5), uint8(1)
memory usage: 1.5 GB


# Write feather-file

In [13]:
%%time

# Save into feather format, about 1.5Gb. 
if write_to_feather == 1:
    df.to_feather(feather_file)

Wall time: 0 ns
